In [ ]:
! pip install pandas datasets transformers

In [ ]:
# ! huggingface-cli login

In [6]:
# load dataset
import pandas as pd
from datasets import Dataset, DatasetDict

df = pd.read_csv("realdonaldtrump.csv")
dataset = Dataset.from_pandas(df)

#print(df.head())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# tokenize the text we want to fine tune on
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
tokenizer.pad_token = tokenizer.eos_token

def preprocess_data(examples):
    return tokenizer(
        examples["content"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(preprocess_data, batched=True)

print(tokenized_dataset[0])


Map: 100%|██████████| 43352/43352 [00:06<00:00, 6445.92 examples/s]

{'id': 1698308935, 'link': 'https://twitter.com/realDonaldTrump/status/1698308935', 'content': 'Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!', 'date': '2009-05-04 13:54:25', 'retweets': 510, 'favorites': 917, 'mentions': None, 'hashtags': None, 'input_ids': [128000, 3513, 2771, 311, 26306, 304, 323, 3821, 9641, 3420, 389, 36931, 13120, 449, 6941, 27757, 1543, 439, 568, 18911, 279, 7054, 18165, 1796, 18396, 0, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 12

In [ ]:
# format data

def format_data(example):
    example["input_ids"] = tokenizer.encode(example["text"], truncation=True, max_length=512)
    example["labels"] = example["input_ids"].copy()
    return example

formatted_dataset = tokenized_dataset.map(format_data, remove_columns=["text"])
print(formatted_dataset["train"][0])

{'id': 1698308935, 'link': 'https://twitter.com/realDonaldTrump/status/1698308935', 'content': 'Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!', 'date': '2009-05-04 13:54:25', 'retweets': 510, 'favorites': 917, 'mentions': None, 'hashtags': None, 'input_ids': [128000, 3513, 2771, 311, 26306, 304, 323, 3821, 9641, 3420, 389, 36931, 13120, 449, 6941, 27757, 1543, 439, 568, 18911, 279, 7054, 18165, 1796, 18396, 0, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 12